# install

### Install Lightning 2.0 

In [1]:
%pip install -qqq lightning
%pip install -qqq timm torchmetrics wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.5/553.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# import

In [2]:
import os
import gc

import time
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# torchvision
import torchvision
from torchvision import transforms, datasets

# wandb
import wandb
from lightning.pytorch.loggers import WandbLogger

# torchmetrics
import torchmetrics

# timm
import timm
from timm import create_model

# import Pytorch Lightning 2.0 
import lightning as L

# Fabric
from lightning.fabric import Fabric

# Config

In [3]:
config= {'model_name':'resnet18', 
         'seed': 2023,
         'bs': 32,
         'n_epochs': 10,
         'lr': 1e-3,
         'is_compiled': True, 
         'mode': 'default',
         'strategy': "auto",
         }

# Ramdom SEED

In [4]:
L.seed_everything(config['seed'])

INFO: Global seed set to 2023
INFO:lightning.fabric.utilities.seed:Global seed set to 2023


2023

# Fabric launch

In [5]:
# fabric = Fabric(accelerator="cuda", devices=8, strategy="ddp")
fabric = Fabric(accelerator= "auto", devices= "auto", strategy="auto")
fabric.launch()

# LitModel

In [6]:
class LitModel(L.LightningModule):
    def __init__(self, model_name, is_compiled, mode =None, batch_size = 32, lr = 1e-3):
        super().__init__()
        self.model = self.get_model(model_name, is_compiled, mode)
        self.batch_size = batch_size
        self.lr = lr

        # Lightning 2.0
        self.losses = []

        # Loss Function
        self.criterion = nn.CrossEntropyLoss()

        # torchmetrics modules
        self.f1 = torchmetrics.F1Score(task="multiclass", num_classes = 10)

    def forward(self, x):
        return self.model(x)

    def get_model(self, model_name, is_compiled, mode):
        
        model = create_model(model_name, pretrained=True, num_classes=10)

        if is_compiled:
            print(f"model_name: {model_name} | Compiled?: {is_compiled} | Compiled MODE: {mode}")
            compiled_model = torch.compile(model, mode = mode) #, backend="aot_ts_nvfuser") # backend="inductor")
            return compiled_model

        else:

            print(f"model_name: {model_name}")
            return model

    def shared_step(self, batch, mode):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        self.losses.append(loss)

        # F1 Score
        f1 = self.f1(logits, y)

        return {'loss': loss, "f1": f1}
        
    def shared_epoch_end(self, mode):

        loss = torch.stack(self.losses).mean().item()

        metrics = {f'{mode}_loss_epoch': loss, f'{mode}_f1_epoch': self.f1.compute()}
        
        self.log_dict(metrics, prog_bar=True)

        return {'loss': loss, "f1": self.f1.compute()}

        # Reset
        self.f1.reset()
        self.losses.clear()

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, "valid")
    
    # https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#validation-epoch-level-metrics
    def on_validation_epoch_end(self):
        return self.shared_epoch_end(mode = "valid")

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch,"train")

    # reference: https://github.com/Lightning-AI/lightning/pull/16520
    def on_train_epoch_end(self):
        return self.shared_epoch_end(mode = "train")

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def train_dataloader(self):
        # Return your dataloader for training
        train_set = datasets.CIFAR10(root="~/data", train=True, download=True, transform=transforms.ToTensor())
        train_loader = torch.utils.data.DataLoader(train_set, batch_size= self.batch_size, shuffle=True, num_workers= os.cpu_count())
        return train_loader
    
    def val_dataloader(self):
        val_set = datasets.CIFAR10(root="~/val_data", train=False, download=True, transform=transforms.ToTensor())
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=self.batch_size, shuffle=False, num_workers= os.cpu_count())
        return val_loader

In [7]:
model = LitModel(model_name = config['model_name'],
                 is_compiled = config["is_compiled"],
                 mode = config["mode"],                  
                 batch_size = config['bs'],
                 lr = config['lr'])

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


model_name: resnet18 | Compiled?: True | Compiled MODE: default


# Wandb Logger

In [8]:
# Reference: https://lightning.ai/docs/pytorch/stable/visualize/logging_intermediate.html?highlight=wandb_logger

wandb_logger = WandbLogger( project= 'fabric_test', 
                            config=config,
                            job_type='Train',
                            name=  "[compiled] lightning_2",
                            anonymous='must')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Trainer

In [9]:
trainer = L.Trainer(accelerator="auto", 
                    devices = -1, 
                    max_epochs= config['n_epochs'],
                    logger = wandb_logger,
                    strategy = config["strategy"]
                    )

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


# Trainer fit!

In [10]:
trainer.fit(model)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type              | Params
------------------------------------------------
0 | model     | OptimizedModule   | 11.2 M
1 | criterion | CrossEntropyLoss  | 0     
2 | f1        | MulticlassF1Score | 0     
------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name      | Type              | Params
------------------------------------------------
0 | model     | OptimizedModule   | 11.2 M
1 | criterion | CrossEntropyLoss  | 0     
2 | f1        | MulticlassF1Score | 0     
------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]


100%|██████████| 170498071/170498071 [00:05<00:00, 29018166.74it/s]


Extracting /root/val_data/cifar-10-python.tar.gz to /root/val_data


100%|██████████| 170498071/170498071 [00:06<00:00, 28322556.85it/s]


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# Finished

In [11]:
wandb.finish()

epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
train_f1_epoch,▁▃▄▅▆▆▇▇██
train_loss_epoch,█▆▅▄▃▃▂▂▁▁
trainer/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
valid_f1_epoch,▁▃▄▅▆▆▇▇██
valid_loss_epoch,█▆▅▄▃▃▂▂▁▁
epoch,9
train_f1_epoch,0.8316
train_loss_epoch,0.5083
trainer/global_step,15629
valid_f1_epoch,0.8316
